# <center>Домашнее задание № 10 (демо)
## <center> Прогнозирование задержек вылетов

Ваша задача – побить единственный бенчмарк в [соревновании](https://www.kaggle.com/c/flight-delays-2017) на Kaggle Inclass. Подробных инструкций не будет, будет только тезисно описано, как получен этот бенчмарк. Конечно, с помощью Xgboost. Надеюсь, на данном этапе курса вам достаточно бросить полтора взгляда на данные, чтоб понять, что это тот тип задачи, в которой затащит Xgboost. Но проверьте еще Catboost.


In [25]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from scipy.sparse import coo_matrix, hstack
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline
from matplotlib import pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from catboost import CatBoostClassifier

In [3]:
train = pd.read_csv('https://raw.githubusercontent.com/Yorko/mlcourse.ai/master/data/flight_delays_train.csv')
test = pd.read_csv('https://raw.githubusercontent.com/Yorko/mlcourse.ai/master/data/flight_delays_test.csv')

In [4]:
train.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min
0,c-8,c-21,c-7,1934,AA,ATL,DFW,732,N
1,c-4,c-20,c-3,1548,US,PIT,MCO,834,N
2,c-9,c-2,c-5,1422,XE,RDU,CLE,416,N
3,c-11,c-25,c-6,1015,OO,DEN,MEM,872,N
4,c-10,c-7,c-6,1828,WN,MDW,OMA,423,Y


In [5]:
train[train['dep_delayed_15min'] == 'Y'].groupby(['DayOfWeek']).count()

,Month,DayofMonth,DepTime,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min
DayOfWeek,,,,,,,,
c-1,2774,2774,2774,2774,2774,2774,2774,2774
c-2,2451,2451,2451,2451,2451,2451,2451,2451
c-3,2567,2567,2567,2567,2567,2567,2567,2567
c-4,3117,3117,3117,3117,3117,3117,3117,3117
c-5,3285,3285,3285,3285,3285,3285,3285,3285
c-6,2145,2145,2145,2145,2145,2145,2145,2145
c-7,2705,2705,2705,2705,2705,2705,2705,2705


Итак, надо по времени вылета самолета, коду авиакомпании-перевозчика, месту вылета и прилета и расстоянию между аэропортами вылета и прилета предсказать задержку вылета более 15 минут. В качестве простейшего бенчмарка возьмем логистическую регрессию и два признака, которые проще всего взять: `DepTime` и `Distance`. У такой модели результат – ***0.68202*** на LB. 

In [ ]:
X_train, y_train = train[['Distance', 'DepTime']].values, train['dep_delayed_15min'].map({'Y': 1, 'N': 0}).values
X_test = test[['Distance', 'DepTime']].values

X_train_part, X_valid, y_train_part, y_valid = train_test_split(X_train, 
                                                                y_train, 
                                                                test_size=0.3, 
                                                                random_state=17)

In [ ]:
logit = LogisticRegression(random_state=17)

logit.fit(X_train_part, y_train_part)
roc_auc_score(y_valid, logit.predict_proba(X_valid)[:, 1])

0.6795697123357751

In [ ]:
xgb_model = XGBClassifier(seed=17)

xgb_model.fit(X_train_part, y_train_part)
roc_auc_score(y_valid, xgb_model.predict_proba(X_valid)[:, 1])

0.6974401153652741

Как был получен бенчмарк в соревновании:
- Признаки `Distance` и  `DepTime` брались без изменений
- Создан признак "маршрут" из исходных `Origin` и `Dest`
- К признакам `Month`, `DayofMonth`, `DayOfWeek`, `UniqueCarrier` и "маршрут" применено OHE-преобразование (`LabelBinarizer`)
- Выделена отложенная выборка
- Обучалась логистическая регрессия и градиентный бустинг (xgboost), гиперпараметры бустинга настраивались на кросс-валидации, сначала те, что отвечают за сложность модели, затем число деревьев фиксировалось равным 500 и настраивался шаг градиентного спуска
- С помощью `cross_val_predict` делались прогнозы обеих моделей на кросс-валидации (именно предсказанные вероятности), настраивалась линейная смесь ответов логистической регрессии и градиентного бустинга вида $w_1 * p_{logit} + (1 - w_1) * p_{xgb}$, где $p_{logit}$ – предсказанные логистической регрессией вероятности класса 1, $p_{xgb}$ – аналогично. Вес $w_1$ подбирался вручную. 
- В качестве ответа для тестовой выборки бралась аналогичная комбинация ответов двух моделей, но уже обученных на всей обучающей выборке.

Описанный план ни к чему не обязывает – это просто то, как решение получил автор задания. Возможно, мы не захотите следовать намеченному плану, а добавите, скажем, пару хороших признаков и обучите лес из тысячи деревьев.

Удачи!

## Baseline as described above

In [26]:
train = pd.read_csv('https://raw.githubusercontent.com/Yorko/mlcourse.ai/master/data/flight_delays_train.csv')
test = pd.read_csv('https://raw.githubusercontent.com/Yorko/mlcourse.ai/master/data/flight_delays_test.csv')

In [27]:
y_train = train['dep_delayed_15min'].map({'Y': 1, 'N': 0})
train = train.drop(['dep_delayed_15min'], axis=1)

In [28]:
idx = train.shape[0]
df = pd.concat([train, test],ignore_index=True)

In [29]:
df['Route'] = df['Origin'] + "_" + df['Dest']
df = df.drop(['Origin', 'Dest'], axis=1)
cat_df = df

In [ ]:
ohe = OneHotEncoder()
scaler = MinMaxScaler()
df[['DepTime', 'Distance']] = scaler.fit_transform(df[['DepTime', 'Distance']])

In [ ]:
df_ohe = ohe.fit_transform(df[['Month', 'DayofMonth', 'DayOfWeek', 
                               'UniqueCarrier', 'Route']])

In [ ]:
df_new = hstack([df[['DepTime', 'Distance']], df_ohe]).tocsr()

In [ ]:
X_train = df_new[:idx]
X_test = df_new[idx:]

In [ ]:
X_train_part, X_valid, y_train_part, y_valid = train_test_split(X_train, 
                                                                y_train, 
                                                                test_size=0.3, 
                                                                random_state=17)

In [41]:
folds = 5
skf = StratifiedKFold(n_splits=folds, shuffle = True)

In [ ]:
c_values = np.logspace(-2, 2, 10)
logit = LogisticRegression()
logit_grid_searcher = GridSearchCV(estimator=logit, param_grid={'C': c_values},
                                  scoring='roc_auc', n_jobs=-1, cv=skf)

In [ ]:
%%time
logit_grid_searcher.fit(X_train_part, y_train_part)

CPU times: user 1.7 s, sys: 175 ms, total: 1.87 s
Wall time: 46.9 s


GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=None, shuffle=True),
             error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='lbfgs',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'C': array([1.00000000e-02, 2.78255940e-02, 7.74263683e-02, 2.15443469e-01,
       5.99484250e-01, 1.66810054e+00, 4.64158883e+00, 1.29154967e+01,
       3.59381366e+01, 1.00000000e+02])},
             pre_dispatch='2*n_jobs', refit=True, return_train_scor

In [ ]:
logit_grid_searcher.best_params_

{'C': 0.0774263682681127}

In [ ]:
best_logit = logit_grid_searcher.best_estimator_

In [ ]:
best_logit.fit(X_train_part, y_train_part)
roc_auc_score(y_valid, best_logit.predict_proba(X_valid)[:, 1])

0.6990655550620921

In [ ]:
# A parameter grid for XGBoost
params = {
        'min_child_weight': [10, 15, 20],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.4, 0.6, 0.8, 1.0],
        'colsample_bytree': [0.1, 0.5, 1.0],
        'max_depth': [3, 5, 10]
        }

In [ ]:
xgb = XGBClassifier(objective='binary:logistic', silent=True, nthread=-1)

In [42]:
param_comb = 5

In [ ]:
random_search = GridSearchCV(xgb, param_grid==params, 
                                   n_iter=param_comb, scoring='roc_auc', 
                                   n_jobs=-1, cv=skf, 
                                   verbose=0)

In [ ]:
%%time
random_search.fit(X_train_part,y_train_part)

CPU times: user 9.88 s, sys: 87.2 ms, total: 9.96 s
Wall time: 2min 46s


RandomizedSearchCV(cv=StratifiedKFold(n_splits=10, random_state=None, shuffle=True),
                   error_score=nan,
                   estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                           colsample_bylevel=1,
                                           colsample_bynode=1,
                                           colsample_bytree=1, gamma=0,
                                           learning_rate=0.1, max_delta_step=0,
                                           max_depth=3, min_child_weight=1,
                                           missing=None, n_estimators=100,
                                           n_jobs=1, nthread=-1,
                                           objective='bin...
                                           reg_lambda=1, scale_pos_weight=1,
                                           seed=None, silent=True, subsample=1,
                                           verbosity=1),
                   iid='deprecat

In [ ]:
random_search.best_params_

{'colsample_bytree': 1.0,
 'gamma': 0.5,
 'max_depth': 10,
 'min_child_weight': 15,
 'subsample': 0.8}

In [ ]:
xgb = XGBClassifier(n_estimators = 500, objective='binary:logistic',
                    subsample = random_search.best_params_['subsample'],
                    min_child_weight = random_search.best_params_['min_child_weight'],
                    max_depth = random_search.best_params_['max_depth'],
                    gamma = random_search.best_params_['gamma'],
                    colsample_bytree = random_search.best_params_['colsample_bytree'],
                    nthread=-1)

In [ ]:
new_params = {'learning_rate' : np.arange(0.1, 1, 0.1)}

In [ ]:
random_search2 = GridSearchCV(xgb, param_grid=new_params, 
                                   n_iter=param_comb, scoring='roc_auc', 
                                   n_jobs=-1, cv=skf.split(X_train_part,y_train_part), 
                                   verbose=0)

In [ ]:
%%time
random_search2.fit(X_train_part,y_train_part)

In [ ]:
random_search2.best_params_

In [ ]:
best_xgb = random_search2.best_estimator_

In [ ]:
best_xgb.fit(X_train_part, y_train_part)
roc_auc_score(y_valid, best_xgb.predict_proba(X_valid)[:, 1])

In [ ]:
w = np.arange(0.05, 1, 0.05)

In [ ]:
d = {}
for i in w:
    d[i] = roc_auc_score(y_valid, i * best_logit.predict_proba(X_valid)[:, 1] 
                         + (1 - i) * best_xgb.predict_proba(X_valid)[:, 1])

In [ ]:
w1 = max(d, key=d.get)
print(w1)

In [ ]:
pred_prob = w1*best_logit.predict_proba(X_test)[:, 1] + (1 - w1)*best_xgb.predict_proba(X_test)[:, 1]

In [ ]:
pd.Series(pred_prob, 
          name='dep_delayed_15min').to_csv('logit+xgb.csv', 
                                           index_label='id', 
                                           header=True)

In [30]:
from catboost import CatBoostClassifier

In [32]:
X_train_cat = cat_df[:idx]
X_test_cat = cat_df[idx:]

In [33]:
X_train_cat_part, cat_valid_df, y_train_part, y_valid = train_test_split(X_train_cat, y_train, 
                                                                  test_size=.3)

In [34]:
cat_features_idx = np.where(X_train_cat_part.dtypes == 'object')[0].tolist()

In [ ]:
ctb = CatBoostClassifier(loss_function='Logloss', cat_features=cat_features_idx, task_type='GPU')

In [35]:
%%time
ctb.fit(X_train_cat_part, y_train_part,
        cat_features=cat_features_idx,silent=True)

CPU times: user 5min 36s, sys: 22.2 s, total: 5min 58s
Wall time: 1min 37s


In [36]:
roc_auc_score(y_valid, ctb.predict_proba(cat_valid_df)[:, 1])

0.7520995903657917

In [38]:
cat_grid = {'learning_rate': [0.02, 0.04, 0.06, 0.08, 1],
        'depth': [1, 2, 3, 4, 5],
        'l2_leaf_reg': [5, 7, 9]}

In [44]:
cat_search = GridSearchCV(ctb, param_distributions=cat_grid, 
                                   n_iter=param_comb, scoring='roc_auc', 
                                   n_jobs=-1, cv=skf, 
                                   verbose=0)

In [46]:
cat_search.fit(X_train_cat_part, y_train_part,
        cat_features=cat_features_idx,silent=True)

KeyboardInterrupt: 

In [ ]:
best_cat = cat_search.best_estimator_
cat_search.best_parameters_

In [ ]:
best_cat.fit(X_train_cat_part, y_train_part, cat_features=cat_features_idx,silent=True)
roc_auc_score(y_valid, ctb.predict_proba(cat_valid_df)[:, 1])

In [ ]:
d = {}
for i in w:
    d[i] = roc_auc_score(y_valid, i * best_cat.predict_proba(X_vacat_valid_dfid)[:, 1] 
                         + (1 - i) * best_xgb.predict_proba(X_valid)[:, 1])

In [ ]:
w1 = max(d, key=d.get)
print(w1)

In [ ]:
pred_prob = w1*best_cat.predict_proba(X_test_cat)[:, 1] + (1 - w1)*best_xgb.predict_proba(X_test)[:, 1]

In [ ]:
pd.Series(pred_prob, 
          name='dep_delayed_15min').to_csv('best_overal.csv', 
                                           index_label='id', 
                                           header=True)